<a href="https://colab.research.google.com/github/MaSaKaIV/QAwithOwnLLM/blob/main/qa_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# パッケージのインストール
!pip install llama-index
!pip install transformers accelerate bitsandbytes
!pip install sentencepiece einops sentence_transformers
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets
!pip install pypdf

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-icioaps5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-icioaps5
  Resolved https://github.com/huggingface/peft.git to commit 85013987aa82aa1af3da1236b6902556ce3e483e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from llama_index import SimpleDirectoryReader

# ドキュメントの読み込み
documents = SimpleDirectoryReader("/content/drive/MyDrive/Workspace/QA/data").load_data()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
import torch

# トークナイザーとモデルの準備
model_id = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
tokenizer_model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)

# パイプラインの準備
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256
)
pipe.tokenizer.pad_token_id = pipe.model.config.pad_token_id
pipe.tokenizer.eos_token_id=pipe.model.config.eos_token_id,

# LLMの準備
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

# 埋め込みモデルの準備
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="oshizo/sbert-jsnli-luke-japanese-base-lite")
)

In [5]:
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser

text_splitter = TokenTextSplitter(
    separator=" ",
    chunk_size=514,
    chunk_overlap=20,
    tokenizer=tokenizer
)

# ノードパーサーの準備
node_parser = SimpleNodeParser(text_splitter=text_splitter)

In [6]:
from llama_index import ServiceContext

# サービスコンテキストの準備
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
)

In [7]:
from llama_index import VectorStoreIndex

# インデックスの作成
index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
)

In [8]:
from llama_index.prompts.prompts import QuestionAnswerPrompt

# QAテンプレートの準備
qa_template = QuestionAnswerPrompt("""<s>[INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

{query_str} [/INST]
""")

In [9]:
# クエリエンジンの作成
query_engine = index.as_query_engine(
    similarity_top_k=3,
    text_qa_template=qa_template,
)

In [10]:
response = query_engine.query( "クラウドセキュリティのガイドラインについて要約して")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


 クラウドセキュリティのガイドラインは、クラウドサービスの利用におけるセキュリティの考え方や対策を紹介したものです。

ガイドラインでは、クラウドサービスの利用時におけるセキュリティ対策として、以下のような項目が挙げられています。

- クラウドサービスプロバイダーの選定
- データの暗号化
- アクセス管理
- セキュリティ対策の継続的な改善
